In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import _tree
import pickle

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.tree import export_text

# ============================================================================
# 1. CHARGEMENT ET ENCODAGE DES DONNÉES
# ============================================================================

In [2]:
df = pd.read_csv("../DATAS/ANSTAT2021_clusters_PC.csv")

In [3]:
# Sauvegarde des données originales pour référence
df_original = df.copy()

In [4]:
# Variables catégorielles à encoder
cat_vars = ['sex', 'marital_status', 'city', 'milieu_resid', 'region_name']

# Dictionnaires pour stocker les encoders et les mappings inversés
label_encoders = {}
inverse_mappings = {}  # Pour reconvertir code → label original

for col in cat_vars:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

    # Créer le mapping inverse : {code_numérique: label_original}
    inverse_mappings[col] = dict(enumerate(le.classes_))


print("✓ Encodage terminé")
print(f"Mappings inversés disponibles pour: {list(inverse_mappings.keys())}")
df

✓ Encodage terminé
Mappings inversés disponibles pour: ['sex', 'marital_status', 'city', 'milieu_resid', 'region_name']


,cluster,age_num,sex,marital_status,city,milieu_resid,region_name,bancarise
0,17,29,0,0,1,1,1,1
1,12,17,0,0,1,1,1,0
2,1,15,1,0,1,1,1,0
3,12,12,0,0,1,1,1,0
4,22,34,0,2,1,1,1,0
...,...,...,...,...,...,...,...,...
64469,89,11,1,0,395,0,23,0
64470,89,7,1,0,395,0,23,0
64471,89,10,1,0,395,0,23,0
64472,89,4,1,0,395,0,23,0


# ============================================================================
# 2. PRÉPARATION DES DONNÉES POUR L'ENTRAÎNEMENT
# ============================================================================

In [5]:
y = df['cluster']
X = df.drop(columns=['cluster'])

In [6]:
features = X.columns.tolist()
features

['age_num',
 'sex',
 'marital_status',
 'city',
 'milieu_resid',
 'region_name',
 'bancarise']

In [7]:
# (Optionnel) split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In [8]:
print(f"\n✓ Données préparées:")
print(f"  - Features: {features}")
print(f"  - Train: {len(X_train)}, Test: {len(X_test)}")


✓ Données préparées:
  - Features: ['age_num', 'sex', 'marital_status', 'city', 'milieu_resid', 'region_name', 'bancarise']
  - Train: 45131, Test: 19343


# ================================================================
# 3. Entraîner un arbre de décision pour "expliquer" les clusters
# ================================================================

In [9]:
tree_clf = DecisionTreeClassifier(
    max_depth=4,   # limite la profondeur pour garder des règles lisibles
    min_samples_leaf=30,  # évite des règles sur 2-3 individus
    random_state=42
)

In [10]:
tree_clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=4, min_samples_leaf=30, random_state=42)

In [11]:
print("Score (accuracy) de l'arbre sur le test :", tree_clf.score(X_test, y_test))

Score (accuracy) de l'arbre sur le test : 0.370263144289924


In [12]:
param_grid = {
    "max_depth": [3, 4, 5, 6, 8, None],
    "min_samples_leaf": [1, 5, 10, 20, 30],
    "criterion": ["gini", "entropy"]
}

dt = DecisionTreeClassifier(random_state=42)

In [13]:
grid = GridSearchCV(
    dt,
    param_grid,
    cv=5,
    scoring="accuracy",
    n_jobs=-1
)
grid.fit(X_train, y_train)

best_dt = grid.best_estimator_
print(grid.best_params_)

{'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1}


In [15]:
y_pred = best_dt.predict(X_test)

print("Accuracy :", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))   # precision, recall, f1 par cluster

Accuracy : 0.9925554464147237
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        97
           1       1.00      1.00      1.00       284
           2       0.97      1.00      0.99       107
           3       0.98      1.00      0.99       433
           4       1.00      1.00      1.00        84
           5       1.00      1.00      1.00        63
           6       1.00      0.99      1.00       671
           7       1.00      0.98      0.99       275
           8       0.98      0.98      0.98       275
           9       0.99      0.99      0.99       260
          10       1.00      0.99      1.00       235
          11       0.99      0.98      0.99       144
          12       1.00      0.99      0.99       299
          13       0.99      0.99      0.99       209
          14       1.00      1.00      1.00       410
          15       1.00      1.00      1.00       443
          16       0.99      1.00      1.00       1

In [16]:
print(confusion_matrix(y_test, y_pred))

[[ 97   0   0 ...   0   0   0]
 [  0 283   0 ...   0   0   0]
 [  0   0 107 ...   0   0   0]
 ...
 [  0   0   0 ... 156   0   0]
 [  0   0   0 ...   0 151   0]
 [  0   0   0 ...   0   0  79]]


# ============================================================================
# 4. EXTRACTION DES RÈGLES LISIBLES (VALEURS ORIGINALES)
# ============================================================================

In [17]:
def decode_threshold_to_labels(feature, threshold, operator):
    """
    Convertit un threshold numérique en labels originaux
    """
    if feature not in inverse_mappings:
        # Pour age_num ou bancarise (pas de décodage nécessaire)
        if feature == "age_num":
            return f"{int(threshold)} ans"
        return f"{threshold:.2f}"
    
    mapping = inverse_mappings[feature]
    
    # Récupérer tous les codes possibles
    all_codes = sorted(mapping.keys())
    
    if operator == "<=":
        # Codes <= threshold
        selected_codes = [c for c in all_codes if c <= threshold]
    else:  # ">"
        # Codes > threshold
        selected_codes = [c for c in all_codes if c > threshold]
    
    # Convertir en labels originaux
    labels = [mapping[c] for c in selected_codes]
    
    return labels

In [18]:
def format_condition(feature, operator, threshold):
    """
    Formate une condition en langage naturel avec valeurs originales
    Optimise l'affichage en utilisant 'PAS DANS' quand c'est plus court
    """
    decoded = decode_threshold_to_labels(feature, threshold, operator)
    
    if isinstance(decoded, list):
        # Nettoyer les types numpy
        decoded_clean = [str(v).replace('np.int64(', '').replace(')', '') for v in decoded]
        
        if len(decoded_clean) == 0:
            return f"{feature} (aucune valeur)"
        elif len(decoded_clean) == 1:
            return f"{feature} = '{decoded_clean[0]}'"
        else:
            # Calculer si "PAS DANS" serait plus court
            if feature in inverse_mappings:
                all_values = list(inverse_mappings[feature].values())
                all_values_clean = [str(v).replace('np.int64(', '').replace(')', '') for v in all_values]
                excluded = [v for v in all_values_clean if v not in decoded_clean]
                #inclues = [v for v in all_values_clean if v in decoded_clean]
                
                # Si moins de valeurs exclues et que ça représente < 50% du total
                # if len(excluded) > 0 and len(excluded) < len(decoded_clean) and len(excluded) <= 10:
                #     values_str = "', '".join(excluded)
                #     return f"{feature} PAS DANS ['{values_str}']"
            
            # Sinon, afficher toutes les valeurs incluses
            values_str = "', '".join(decoded_clean)
            # print(f"values_str: {values_str}")
            # print(f"decoded: {decoded}")
            # print(f"Include: {inclues}")
            # print(f"Exclude: {excluded}")
            # print("------------------------------------------------------")
            return f"{feature} ∈ ['{values_str}']"
    else:
        return f"{feature} {operator} {decoded}"

In [19]:
def extract_rules_with_original_values(tree_model, feature_names):
    """
    Extrait toutes les règles de l'arbre avec les valeurs originales
    """
    tree = tree_model.tree_
    
    def recurse(node_id=0, current_conditions=None):
        if current_conditions is None:
            current_conditions = []
        
        rules = []
        
        # Feuille : on a trouvé une règle complète
        if tree.feature[node_id] == _tree.TREE_UNDEFINED:
            cluster = int(np.argmax(tree.value[node_id][0]))
            n_samples = int(tree.n_node_samples[node_id])
            rules.append({
                "conditions": current_conditions.copy(),
                "cluster": cluster,
                "n_samples": n_samples
            })
            return rules
        
        # Nœud interne : on continue la récursion
        feature = feature_names[tree.feature[node_id]]
        threshold = tree.threshold[node_id]
        
        # Branche gauche (<=)
        left_conditions = current_conditions + [(feature, "<=", threshold)]
        rules.extend(recurse(tree.children_left[node_id], left_conditions))
        
        # Branche droite (>)
        right_conditions = current_conditions + [(feature, ">", threshold)]
        rules.extend(recurse(tree.children_right[node_id], right_conditions))
        
        return rules
    
    return recurse()

In [20]:
# Extraction des règles
print("\n⏳ Extraction des règles avec valeurs originales...")
rules_raw = extract_rules_with_original_values(best_dt, features)
rules_raw


⏳ Extraction des règles avec valeurs originales...


[{'conditions': [('marital_status', '<=', np.float64(0.5)),
   ('milieu_resid', '<=', np.float64(0.5)),
   ('bancarise', '<=', np.float64(0.5)),
   ('sex', '<=', np.float64(0.5)),
   ('region_name', '<=', np.float64(2.5)),
   ('age_num', '<=', np.float64(27.5))],
  'cluster': 71,
  'n_samples': 388},
 {'conditions': [('marital_status', '<=', np.float64(0.5)),
   ('milieu_resid', '<=', np.float64(0.5)),
   ('bancarise', '<=', np.float64(0.5)),
   ('sex', '<=', np.float64(0.5)),
   ('region_name', '<=', np.float64(2.5)),
   ('age_num', '>', np.float64(27.5)),
   ('age_num', '<=', np.float64(58.0))],
  'cluster': 80,
  'n_samples': 11},
 {'conditions': [('marital_status', '<=', np.float64(0.5)),
   ('milieu_resid', '<=', np.float64(0.5)),
   ('bancarise', '<=', np.float64(0.5)),
   ('sex', '<=', np.float64(0.5)),
   ('region_name', '<=', np.float64(2.5)),
   ('age_num', '>', np.float64(27.5)),
   ('age_num', '>', np.float64(58.0))],
  'cluster': 7,
  'n_samples': 6},
 {'conditions': [('ma

In [21]:
rules_readable = []
for rule in rules_raw:
    # Regrouper les conditions par feature pour fusionner les répétitions
    conditions_by_feature = {}
    
    for feature, op, threshold in rule["conditions"]:
        if feature not in conditions_by_feature:
            conditions_by_feature[feature] = []
        conditions_by_feature[feature].append((op, threshold))
    
    # Fusionner les conditions multiples sur la même variable
    conditions_text = []
    for feature, ops_thresholds in conditions_by_feature.items():
        if len(ops_thresholds) == 1:
            # Une seule condition : format normal
            op, threshold = ops_thresholds[0]
            cond_str = format_condition(feature, op, threshold)
            conditions_text.append(cond_str)
        else:
            # Plusieurs conditions sur la même variable : FUSIONNER
            # Pour les variables catégorielles, calculer l'intersection
            if feature in inverse_mappings:
                all_possible = set(inverse_mappings[feature].values())
                current_set = all_possible.copy()
                
                for op, threshold in ops_thresholds:
                    decoded = decode_threshold_to_labels(feature, threshold, op)
                    if isinstance(decoded, list):
                        current_set &= set(decoded)
                
                # Créer la condition finale avec les valeurs fusionnées
                final_values = sorted(list(current_set))
                if len(final_values) == 0:
                    cond_str = f"{feature} (aucune valeur)"
                elif len(final_values) == 1:
                    cond_str = f"{feature} = '{final_values[0]}'"
                else:
                    values_str = "', '".join([str(v) for v in final_values])
                    cond_str = f"{feature} ∈ ['{values_str}']"
                
                conditions_text.append(cond_str)
            else:
                # Pour les variables numériques (age_num), garder les bornes min/max
                less_equal = [t for o, t in ops_thresholds if o == "<="]
                greater = [t for o, t in ops_thresholds if o == ">"]
                
                if less_equal and greater:
                    min_val = int(max(greater))
                    max_val = int(min(less_equal))
                    cond_str = f"{min_val} < {feature} <= {max_val} ans"
                elif less_equal:
                    max_val = int(min(less_equal))
                    cond_str = f"{feature} <= {max_val} ans"
                elif greater:
                    min_val = int(max(greater))
                    cond_str = f"{feature} > {min_val} ans"
                
                conditions_text.append(cond_str)
    
    rule_text = " ET ".join(conditions_text)
    
    rules_readable.append({
        "Règle_ID": len(rules_readable) + 1,
        "Conditions": rule_text,
        "Cluster_Assigné": rule["cluster"],
        "Nombre_Individus": rule["n_samples"]
    })

rules_df = pd.DataFrame(rules_readable)
print(f"✓ {len(rules_df)} règles extraites")
# rules_df

✓ 804 règles extraites


In [22]:
rules_df_sorted = rules_df.sort_values(
    by=["Cluster_Assigné", "Règle_ID"],
    ascending=[True, True]
).reset_index(drop=True)

rules_df_sorted.head(20)

,Règle_ID,Conditions,Cluster_Assigné,Nombre_Individus
0,61,marital_status = 'Célibataire' ET milieu_resid...,0,179
1,197,marital_status = 'Célibataire' ET milieu_resid...,0,3
2,200,marital_status = 'Célibataire' ET milieu_resid...,0,1
3,205,marital_status = 'Célibataire' ET milieu_resid...,0,24
4,274,marital_status = 'Célibataire' ET milieu_resid...,0,19
5,483,marital_status = 'Veuf(ve)' ET bancarise <= 0....,0,1
6,335,marital_status = 'Célibataire' ET milieu_resid...,1,367
7,338,marital_status = 'Célibataire' ET milieu_resid...,1,11
8,379,marital_status = 'Célibataire' ET milieu_resid...,1,274
9,390,marital_status = 'Célibataire' ET milieu_resid...,1,9


In [23]:
rules_df = rules_df.sort_values(
    by=["Cluster_Assigné", "Règle_ID"]
).reset_index(drop=True)
rules_df

,Règle_ID,Conditions,Cluster_Assigné,Nombre_Individus
0,61,marital_status = 'Célibataire' ET milieu_resid...,0,179
1,197,marital_status = 'Célibataire' ET milieu_resid...,0,3
2,200,marital_status = 'Célibataire' ET milieu_resid...,0,1
3,205,marital_status = 'Célibataire' ET milieu_resid...,0,24
4,274,marital_status = 'Célibataire' ET milieu_resid...,0,19
...,...,...,...,...
799,286,marital_status = 'Célibataire' ET milieu_resid...,95,3
800,295,marital_status = 'Célibataire' ET milieu_resid...,95,6
801,354,marital_status = 'Célibataire' ET milieu_resid...,95,103
802,420,marital_status = 'Célibataire' ET milieu_resid...,95,68


# ============================================================================
# 5. SAUVEGARDE DES RÈGLES ET DU MODÈLE
# ============================================================================

In [24]:
rules_df.to_csv("regles_segmentation_lisibles.csv", index=False, encoding="utf-8")
print("✓ Règles sauvegardées: regles_segmentation_lisibles.csv")

✓ Règles sauvegardées: regles_segmentation_lisibles.csv


In [25]:
# Sauvegarder le modèle et les encoders
with open("modele_clustering.pkl", "wb") as f:
    pickle.dump({
        "model": best_dt,
        "label_encoders": label_encoders,
        "inverse_mappings": inverse_mappings,
        "features": features
    }, f)
print("✓ Modèle sauvegardé: modele_clustering.pkl")

✓ Modèle sauvegardé: modele_clustering.pkl


In [26]:
# Afficher un échantillon de règles
print("\n" + "="*80)
print("EXEMPLE DE RÈGLES EXTRAITES (5 premières):")
print("="*80)
print(rules_df.head(10).to_string(index=False))


EXEMPLE DE RÈGLES EXTRAITES (5 premières):
 Règle_ID                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

# ============================================================================
# 6. FONCTION D'ASSIGNATION POUR NOUVELLES DONNÉES
# ============================================================================

In [27]:
def assigner_nouveaux_individus(csv_path, model_path="modele_clustering.pkl"):
    """
    Assigne des clusters à de nouveaux individus
    
    Args:
        csv_path: Chemin vers le CSV avec les nouvelles données
        model_path: Chemin vers le modèle sauvegardé
    
    Returns:
        DataFrame avec les clusters assignés
    """
    # Charger le modèle et les encoders
    with open(model_path, "rb") as f:
        saved_data = pickle.load(f)
    
    model = saved_data["model"]
    label_encoders = saved_data["label_encoders"]
    features = saved_data["features"]
    
    # Charger les nouvelles données
    new_data = pd.read_csv(csv_path)
    print(f"✓ {len(new_data)} nouveaux individus chargés")
    
    # Encoder les variables catégorielles avec les MÊMES encoders
    new_data_encoded = new_data.copy()
    for col in label_encoders.keys():
        if col in new_data_encoded.columns:
            le = label_encoders[col]
            # Gérer les valeurs inconnues
            new_data_encoded[col] = new_data_encoded[col].apply(
                lambda x: le.transform([x])[0] if x in le.classes_ else -1
            )
    
    # Prédiction
    X_new = new_data_encoded[features]
    clusters = model.predict(X_new)
    result = new_data.copy()
    result['Cluster_Assigné'] = clusters
    
    return result


In [33]:
# nouvelles_assignations = assigner_nouveaux_individus("./Paquets/Proxy.csv")
# nouvelles_assignations = assigner_nouveaux_individus("../DATAS/ANSTAT2021_clusters_PC.csv")
nouvelles_assignations = assigner_nouveaux_individus("../DATAS/ANSTAT2021_dataset_AllVars_CLEAN.csv")

print(nouvelles_assignations['Cluster_Assigné'].nunique())
print(nouvelles_assignations['Cluster_Assigné'].unique())
nouvelles_assignations.to_csv("resultats_assignation.csv", index=False)

✓ 64474 nouveaux individus chargés
97
[17 12  1 22 56 94 23 11 33 34 91  3  9 60 81 55 92  8 10 71 88  7 31 13
 80 77 52 21 24 85 15  2 59 53 25 19  6 14 76 64 68 93 62 50 44 72 49 90
 96 83 84 57 78 29 36 42 38 32 26 54 40 61  0 74 45  4 30 79 66 48 70 39
 95 28 27 82 58 35 47 16 51 37 65 67  5 87 20 46 89 73 63 86 18 43 69 41
 75]


In [ ]:
print("\n" + "="*80)
print("✓ PROCESSUS TERMINÉ")
print("="*80)
print("\nFichiers générés:")
print("  1. regles_segmentation_lisibles.csv - Règles en langage naturel")
print("  2. modele_clustering.pkl - Modèle + encoders pour nouvelles données")
print("\nPour assigner de nouveaux individus:")
print("  nouvelles = assigner_nouveaux_individus('votre_fichier.csv')")


✓ PROCESSUS TERMINÉ

Fichiers générés:
  1. regles_segmentation_lisibles.csv - Règles en langage naturel
  2. modele_clustering.pkl - Modèle + encoders pour nouvelles données

Pour assigner de nouveaux individus:
  nouvelles = assigner_nouveaux_individus('votre_fichier.csv')
